In [1]:
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [99]:
def bi_normal(x1, x2, mu1, mu2, s1, s2, rho,M):

    x1 = x1.contiguous().view(-1, 1).expand(x1.size(0), M)
    x2 = x2.contiguous().view(-1, 1).expand(x1.size(0), M)
    norm1 = x1 - mu1
    norm2 = x2 - mu2
    z = torch.div(norm1, s1).pow(2) + torch.div(norm2, s2).pow(2) - 2 * torch.div(torch.mul(rho, torch.mul(norm1, norm2)), torch.mul(s1,s2))
    coef = torch.exp(-z/(2*(1-rho.pow(2))))
    denom = 2 * F.math.pi * s1 * s2 * torch.sqrt(1-rho.pow(2))
    result = torch.div(coef, denom)
    return result

def Lr(z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_pen_logits, x1, x2, pen, s, M, batch_size, N_max):
    
    indices = []
    for i in range(len(s)): indices += [a+200*i for a in range(s[i])]
    #indices = torch.LongTensor(indices)
    ls = bi_normal(x1[indices,], x2[indices,], z_mu1[indices,], z_mu2[indices,], z_sigma1[indices,], z_sigma2[indices,], z_corr[indices,],M)
    ls = torch.mul(ls, z_pi[indices,])
    ls = torch.sum(ls, 1, keepdim=True)
    ls = - torch.log(ls + 1e-6)
    ls = torch.sum(ls) / x1.size()[0]
                   
    lp = torch.log(z_pen_logits + 1e-6)

    lp = torch.mul(lp, pen)
    lp = - torch.sum(lp, 1, keepdim=True)
    lp = torch.sum(lp) / x1.size()[0]
    return ls+lp

def Lkl(mu, sigma,  R=0.99999, KL_min = 0.2):
    eta_step = 1 - (1- 0.01) * R
    Lkl = -0.5 * torch.mean(1 + sigma - mu.pow(2) - sigma.exp())                 
    return 0.5 * eta_step * torch.max(Lkl, Variable(torch.FloatTensor([0.2])).cuda())


class Encoder(nn.Module):
    
    def __init__(self, strokeSize, batchSize,  Nhe, Nhd, Nz,dropout=0.1):
        super(Encoder, self).__init__()
        self.Nz = Nz
        self.Nhe = Nhe
        self.Nhd = Nhd
        self.cell = nn.LSTM(strokeSize, Nhe, 1, dropout=dropout, bidirectional=True, batch_first=True)
        self.mu = nn.Linear(Nhe*2, Nz)
        self.sigma = nn.Linear(Nhe*2, Nz)
        self.h0 = nn.Linear(Nz, Nhd*2) # returns h0 and c0
        self.train()
        #print(Nh)

    def forward(self, x):
        _, (hn, cn) = self.cell(x)
        hn = Variable(torch.cat((hn.data[0],hn.data[1]),1)).cuda()
        sigma = self.sigma(hn)
        mu = self.mu(hn)
        s = sigma
        sigma = torch.exp( sigma * 0.5)
        N = Variable(torch.normal(torch.zeros(self.Nz),torch.ones(self.Nz))).cuda()
        z = mu + sigma * N
        return F.tanh(self.h0(z)), z, mu, s
        
        
class Decoder(nn.Module):
    
    def __init__(self, strokeSize, batchSize,  Nhe, Nhd, Nz, Ny,dropout=0.1):
        super(Decoder, self).__init__()
        self.Nhe = Nhe
        self.Nhd = Nhd
        self.Nz = Nz
        self.batchSize = batchSize
        self.cell = nn.LSTM(strokeSize+Nz, Nhd, 1, dropout=dropout, batch_first=True)
        self.y = nn.Linear(Nhd, Ny)
    
    def forward(self, x, h0, c0):
        h0 = h0.view(1, h0.size()[0], h0.size()[1])
        c0 = c0.view(1, c0.size()[0], c0.size()[1])
        output, (hn, cn) = self.cell(x, (h0, c0))
        output = output.contiguous().view(-1, self.Nhd)
        y = self.y(output)
        return y
                   
    def predict(self, x, h, c, z, M):
        out = []
        tmp = 0.5
        new_input = torch.cat((x[:, 0, :].contiguous().view(1, 1, 5), z.view(1, 1, self.Nz)), 2)

        h = h.view(1, h.size()[0], h.size()[1])
        c = c.view(1, c.size()[0], c.size()[1])
        
        for i in range(0,200):
            y, (h, c) = self.cell(new_input, (h, c))
            output = y.contiguous().view(-1, self.Nhd)
            y = self.y(output)
            z_pen_logits = y[:, -3:]
            z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr = torch.chunk(y[:, :-3], 6, dim=1)
            z_pi = F.softmax(z_pi / tmp)
            z_pen_logits = F.softmax(z_pen_logits / tmp)
            z_sigma1 = torch.exp(z_sigma1)
            z_sigma2 = torch.exp(z_sigma2)
            z_corr = F.tanh(z_corr)

            probs = z_pi.data[0].cpu().numpy()
            probs /= probs.sum()          
            i = np.random.choice(np.arange(0, M), p=probs)
            mean = [z_mu1.data[0][i], z_mu2.data[0][i]]
            cov = [[z_sigma1.data[0][i] * z_sigma1.data[0][i] * tmp, z_corr.data[0][i] * z_sigma1.data[0][i] * z_sigma2.data[0][i]], \
                   [z_corr.data[0][i] * z_sigma1.data[0][i] * z_sigma2.data[0][i], z_sigma2.data[0][i] * z_sigma2.data[0][i] * tmp]]

            x = np.random.multivariate_normal(mean, cov)
            probs = z_pen_logits.data[0].cpu().numpy()
            probs /= probs.sum()
            iPen = np.random.choice(np.arange(0, 3), p=probs)
            pen = [0, 0, 0]
            pen[iPen] = 1
        
            stroke = [x[0], x[1]] + pen
            print(stroke)
            
            if iPen == 2:
                break
            
            out.append(stroke)

            stroke = torch.FloatTensor(stroke)
            stroke = Variable(stroke).cuda()
            new_input = torch.cat((stroke.contiguous().view(1, 1, 5), z.view(1, 1, self.Nz).data), 2)            

            
        return out
        
class SketchRNN(nn.Module):
    
    def __init__(self, strokeSize, batchSize, Nhe, Nhd, Nz, Ny, max_seq_len):
        super(SketchRNN, self).__init__()
        self.batchSize = batchSize
        self.Nhe = Nhe
        self.Nhd = Nhd
        self.max_seq_len = max_seq_len
        self.Nz = Nz
        self.encoder = Encoder(strokeSize, batchSize, Nhe, Nhd, Nz)
        self.decoder = Decoder(strokeSize, batchSize, Nhe, Nhd, Nz, Ny)
        
    def forward(self, x):
        # we don't take the inisiale S0 in the encoder so we do the 1:
        h0, z, mu, sigma = self.encoder(x[:, 1:250+1, :])
        #here we take S0
        new_input = torch.cat((x[:, :250, :], z.view(self.batchSize, 1, self.Nz)\
            .expand(self.batchSize, self.max_seq_len, self.Nz)), 2)        
        y = self.decoder(new_input, h0[:,:self.Nhd].contiguous(), h0[:,self.Nhd:].contiguous())
        #y = self.decoder(new_input, h0[:,:self.Nhd], h0[:,self.Nhd:])
        return y, mu, sigma

    def predict(self, x, M):
        
        h0, z, mu, sigma = self.encoder(x[:, 1:250+1, :])
        y = self.decoder.predict(x[:, :250, :].contiguous(), h0[:,:self.Nhd].contiguous(), h0[:,self.Nhd:].contiguous(), z, M)

In [51]:
import os
import numpy as np
import utils
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from matplotlib import pyplot as plt
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import time
import _pickle as pickle

t1 = time.time()
filename = "cat.npz"
load_data = np.load(filename, encoding = 'latin1')
train_set = load_data['train']
#valid_set = load_data['valid']
#test_set = load_data['test']

nb_steps = 20000
batch_size = 100
max_seq_len=200
M = 20
strokeSize=5
Y_size = 6*M+3
N_he = 256
N_hd = 512
N_z = 128
w_lk = 0.5
lr = 0.001
temperature = 0.1
lr_decay = 0.9999
min_lr = 0.00001
KL_min = 0.2
grad_clipping = 1.0
N_y = 6*M+3

train_set = utils.DataLoader(
      train_set,
      batch_size,
      max_seq_length=max_seq_len)
normalizing_scale_factor = train_set.calculate_normalizing_scale_factor()
train_set.normalize(normalizing_scale_factor)


encoder = Encoder(strokeSize, batch_size, N_he, N_hd, N_z).cuda()
decoder = Decoder(strokeSize, batch_size, N_he, N_hd, N_z, N_y).cuda()

saved_encoder = torch.load('encoderOUR_epoch_9999.pth')
saved_decoder = torch.load('decoderOUR_epoch_9999.pth')
encoder.load_state_dict(saved_encoder)
decoder.load_state_dict(saved_decoder)

encoder_optimizer = optim.Adam(encoder.parameters(), lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr)

cudnn.benchmark = True

for it in range(10001, nb_steps):
    
    encoder_optimizer.zero_grad() 
    decoder_optimizer.zero_grad()
    
    _, x_train, s = train_set.random_batch()    
    x_train = Variable(torch.from_numpy(x_train).type(torch.FloatTensor)).cuda()
    
    h0, z, mu, sigma = encoder(x_train[:, 1:200+1, :])
    
    new_input = torch.cat((x_train[:, :200, :], z.view(batch_size, 1, N_z)\
            .expand(batch_size, max_seq_len, N_z)), 2)        
    y = decoder(new_input, h0[:,:N_hd].contiguous(), h0[:,N_hd:].contiguous())


    z_pen_logits = y[:, -3:]
    z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr = torch.chunk(y[:, :-3], 6, dim=1)
    z_pi = F.softmax(z_pi)
    z_pen_logits = F.softmax(z_pen_logits)
    z_sigma1 = torch.exp(z_sigma1)
    z_sigma2 = torch.exp(z_sigma2)
    z_corr = F.tanh(z_corr)
    targets = x_train[:, 1:200+1, :].contiguous().view(-1, 5)
    x1 = targets[:,0]
    x2 = targets[:,1]
    pen = targets[:,2:]
   
    lr = Lr(z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_pen_logits, x1, x2, pen, s, M, batch_size, max_seq_len)
    L_kl = Lkl(mu, sigma)
    L_kl=  (L_kl + 1e-6)
           
    loss = lr + L_kl
    loss.backward()

    # gradient cliping
    nn.utils.clip_grad_norm(encoder.parameters(), grad_clipping)
    nn.utils.clip_grad_norm(decoder.parameters(), grad_clipping)
    
    decoder_optimizer.step()
    encoder_optimizer.step()
    
    print('epoch',it,'loss',loss.data[0],'LR',lr.data[0],'LKL',L_kl.data[0])

total images <= max_seq_len is 70000
epoch 10001 loss 0.05516577139496803 LR 0.04167141765356064 LKL 0.013494354672729969
epoch 10002 loss 0.1367446333169937 LR 0.12321306020021439 LKL 0.013531566597521305
epoch 10003 loss 0.07978179305791855 LR 0.06628485769033432 LKL 0.013496936298906803
epoch 10004 loss 0.0920165553689003 LR 0.07858417928218842 LKL 0.013432378880679607
epoch 10005 loss 0.11377096176147461 LR 0.10028848797082901 LKL 0.013482475653290749
epoch 10006 loss 0.08652689307928085 LR 0.07303324341773987 LKL 0.013493647798895836
epoch 10007 loss 0.06694462895393372 LR 0.05344432592391968 LKL 0.013500304892659187
epoch 10008 loss 0.04882526397705078 LR 0.03536015748977661 LKL 0.01346510648727417
epoch 10009 loss 0.05929165706038475 LR 0.04582080990076065 LKL 0.013470848090946674
epoch 10010 loss 0.10547234117984772 LR 0.09204168617725372 LKL 0.013430655933916569
epoch 10011 loss 0.05831432715058327 LR 0.04486968368291855 LKL 0.013444644398987293
epoch 10012 loss 0.057310808449

epoch 10097 loss 0.08394687622785568 LR 0.07060690224170685 LKL 0.013339972123503685
epoch 10098 loss 0.03952799737453461 LR 0.02609361708164215 LKL 0.01343438122421503
epoch 10099 loss 0.05535167083144188 LR 0.04201171547174454 LKL 0.013339955359697342
epoch 10100 loss 0.06997255235910416 LR 0.05656944215297699 LKL 0.013403111137449741
epoch 10101 loss 0.057714808732271194 LR 0.04427456110715866 LKL 0.013440248556435108
epoch 10102 loss 0.05878569930791855 LR 0.04541293531656265 LKL 0.013372762128710747
epoch 10103 loss 0.0679640844464302 LR 0.05456497520208359 LKL 0.013399112038314342
epoch 10104 loss 0.09225129336118698 LR 0.07885278761386871 LKL 0.013398502953350544
epoch 10105 loss 0.06867963820695877 LR 0.05526096373796463 LKL 0.01341867633163929
epoch 10106 loss 0.05204019322991371 LR 0.03857635706663132 LKL 0.01346383523195982
epoch 10107 loss 0.015003697946667671 LR 0.0015843212604522705 LKL 0.0134193766862154
epoch 10108 loss 0.027595389634370804 LR 0.014177516102790833 LKL 0

epoch 10194 loss -0.005228759720921516 LR -0.018739208579063416 LKL 0.013510448858141899
epoch 10195 loss 0.07514344155788422 LR 0.061674486845731735 LKL 0.013468953780829906
epoch 10196 loss 0.05271512269973755 LR 0.03923577815294266 LKL 0.013479344546794891
epoch 10197 loss 0.03282979503273964 LR 0.01937256008386612 LKL 0.01345723494887352
epoch 10198 loss 0.057858455926179886 LR 0.04436406493186951 LKL 0.013494390994310379
epoch 10199 loss 0.04100610315799713 LR 0.027486450970172882 LKL 0.01351965218782425
epoch 10200 loss 0.017282400280237198 LR 0.0037785395979881287 LKL 0.01350386068224907
epoch 10201 loss 0.024823620915412903 LR 0.011328659951686859 LKL 0.013494960963726044
epoch 10202 loss 0.06172545254230499 LR 0.048292018473148346 LKL 0.013433435931801796
epoch 10203 loss 0.04431050643324852 LR 0.03079500049352646 LKL 0.013515506871044636
epoch 10204 loss 0.057435907423496246 LR 0.04395154491066933 LKL 0.01348436251282692
epoch 10205 loss 0.04192053899168968 LR 0.0284563526511

epoch 10290 loss 0.08079342544078827 LR 0.06725135445594788 LKL 0.013542070984840393
epoch 10291 loss 0.0412750244140625 LR 0.027721114456653595 LKL 0.013553908094763756
epoch 10292 loss 0.04343457520008087 LR 0.029920075088739395 LKL 0.013514501973986626
epoch 10293 loss 0.044445887207984924 LR 0.030857644975185394 LKL 0.01358824223279953
epoch 10294 loss 0.03832177817821503 LR 0.024766087532043457 LKL 0.013555689714848995
epoch 10295 loss 0.0637897178530693 LR 0.05024648457765579 LKL 0.013543235138058662
epoch 10296 loss 0.060888610780239105 LR 0.04733441025018692 LKL 0.013554202392697334
epoch 10297 loss 0.012705360539257526 LR -0.0008812621235847473 LKL 0.013586622662842274
epoch 10298 loss 0.051793694496154785 LR 0.03821524977684021 LKL 0.013578443787992
epoch 10299 loss 0.03136104717850685 LR 0.017763465642929077 LKL 0.0135975806042552
epoch 10300 loss 0.05771522969007492 LR 0.04417216032743454 LKL 0.01354306936264038
epoch 10301 loss 0.0604785718023777 LR 0.04692542925477028 LKL

epoch 10386 loss 0.049496009945869446 LR 0.03590182960033417 LKL 0.013594179414212704
epoch 10387 loss 0.02161346934735775 LR 0.007930196821689606 LKL 0.013683272525668144
epoch 10388 loss 0.027181047946214676 LR 0.013536825776100159 LKL 0.013644223101437092
epoch 10389 loss 0.00842195376753807 LR -0.005262993276119232 LKL 0.013684947043657303
epoch 10390 loss 0.08056976646184921 LR 0.06691505014896393 LKL 0.013654718175530434
epoch 10391 loss 0.05698394402861595 LR 0.04330943152308464 LKL 0.013674511574208736
epoch 10392 loss 0.05067279189825058 LR 0.037015821784734726 LKL 0.013656971044838428
epoch 10393 loss 0.041336361318826675 LR 0.027691476047039032 LKL 0.013644886203110218
epoch 10394 loss 0.04852990806102753 LR 0.03486037254333496 LKL 0.013669533655047417
epoch 10395 loss 0.04858475923538208 LR 0.03487551584839821 LKL 0.01370924524962902
epoch 10396 loss 0.039523448795080185 LR 0.02585909515619278 LKL 0.01366435457020998
epoch 10397 loss 0.0291958786547184 LR 0.0155230909585952

...

epoch 19612 loss 0.005815910175442696 LR -0.008827835321426392 LKL 0.014643745496869087
epoch 19613 loss 0.03786216676235199 LR 0.023283235728740692 LKL 0.014578932896256447
epoch 19614 loss 0.00841398723423481 LR -0.006129711866378784 LKL 0.014543699100613594
epoch 19615 loss 0.02359181083738804 LR 0.008973784744739532 LKL 0.014618026092648506
epoch 19616 loss -0.006829054094851017 LR -0.02139122039079666 LKL 0.014562166295945644
epoch 19617 loss 0.004540594294667244 LR -0.010084003210067749 LKL 0.014624597504734993
epoch 19618 loss 0.045663703233003616 LR 0.03110475093126297 LKL 0.014558951370418072
epoch 19619 loss 0.0003942064940929413 LR -0.014133840799331665 LKL 0.014528047293424606
epoch 19620 loss 0.00996622908860445 LR -0.0046306997537612915 LKL 0.014596928842365742
epoch 19621 loss 0.004227308556437492 LR -0.010397464036941528 LKL 0.01462477259337902
epoch 19622 loss 0.009150320664048195 LR -0.005431577563285828 LKL 0.014581898227334023
epoch 19623 loss 0.015258979052305222 L

epoch 19706 loss 0.009715144522488117 LR -0.004888452589511871 LKL 0.014603597111999989
epoch 19707 loss 0.029107756912708282 LR 0.014501161873340607 LKL 0.014606595039367676
epoch 19708 loss 0.01249219011515379 LR -0.0021411702036857605 LKL 0.01463336031883955
epoch 19709 loss 0.03490827977657318 LR 0.020342029631137848 LKL 0.014566251076757908
epoch 19710 loss 0.031692590564489365 LR 0.017053738236427307 LKL 0.014638853259384632
epoch 19711 loss 0.00490329135209322 LR -0.009726598858833313 LKL 0.014629890210926533
epoch 19712 loss -0.004558177664875984 LR -0.019198521971702576 LKL 0.014640344306826591
epoch 19713 loss 0.014292478561401367 LR -0.00034159421920776367 LKL 0.01463407278060913
epoch 19714 loss -0.022200416773557663 LR -0.03680860996246338 LKL 0.014608193188905716
epoch 19715 loss 0.016839168965816498 LR 0.002182692289352417 LKL 0.014656477607786655
epoch 19716 loss 0.008325371891260147 LR -0.00632718950510025 LKL 0.014652561396360397
epoch 19717 loss -0.016101613640785217

epoch 19800 loss 0.008270508609712124 LR -0.006285451352596283 LKL 0.014555959962308407
epoch 19801 loss 0.03836648166179657 LR 0.02377474308013916 LKL 0.01459173671901226
epoch 19802 loss -0.021393103525042534 LR -0.035990744829177856 LKL 0.014597641304135323
epoch 19803 loss 0.00927015207707882 LR -0.005361869931221008 LKL 0.014632022008299828
epoch 19804 loss -0.012706432491540909 LR -0.027272023260593414 LKL 0.014565590769052505
epoch 19805 loss 0.051981568336486816 LR 0.037419162690639496 LKL 0.014562404714524746
epoch 19806 loss -0.0064825452864170074 LR -0.021093323826789856 LKL 0.014610778540372849
epoch 19807 loss 0.008406803011894226 LR -0.006145156919956207 LKL 0.014551959931850433
epoch 19808 loss -0.016958074644207954 LR -0.03143482655286789 LKL 0.014476751908659935
epoch 19809 loss 0.023697467520833015 LR 0.009064510464668274 LKL 0.014632957056164742
epoch 19810 loss 0.01611553132534027 LR 0.0014718696475028992 LKL 0.014643661677837372
epoch 19811 loss 0.03880368918180466

epoch 19894 loss -0.019434021785855293 LR -0.033970482647418976 LKL 0.014536460861563683
epoch 19895 loss 0.04756535217165947 LR 0.032955102622509 LKL 0.014610250480473042
epoch 19896 loss 0.021104276180267334 LR 0.006502158939838409 LKL 0.014602117240428925
epoch 19897 loss -0.03201053664088249 LR -0.046631693840026855 LKL 0.014621157199144363
epoch 19898 loss 0.011340312659740448 LR -0.0033066198229789734 LKL 0.014646932482719421
epoch 19899 loss 0.00881645455956459 LR -0.005713209509849548 LKL 0.014529664069414139
epoch 19900 loss 0.059799883514642715 LR 0.04513810575008392 LKL 0.014661776833236217
epoch 19901 loss -0.006010447628796101 LR -0.020662903785705566 LKL 0.014652456156909466
epoch 19902 loss -0.012050499208271503 LR -0.02662036567926407 LKL 0.014569866470992565
epoch 19903 loss 0.012570267543196678 LR -0.0020312294363975525 LKL 0.01460149697959423
epoch 19904 loss 0.0031167175620794296 LR -0.011498905718326569 LKL 0.014615623280405998
epoch 19905 loss 0.007532700896263122

epoch 19988 loss -0.00751730240881443 LR -0.022114410996437073 LKL 0.014597108587622643
epoch 19989 loss 0.025652915239334106 LR 0.011076733469963074 LKL 0.014576180838048458
epoch 19990 loss 0.007675365544855595 LR -0.00699479877948761 LKL 0.014670164324343204
epoch 19991 loss 0.019217653200030327 LR 0.004628174006938934 LKL 0.014589479193091393
epoch 19992 loss 0.03688130900263786 LR 0.022246554493904114 LKL 0.01463475450873375
epoch 19993 loss -0.03889653831720352 LR -0.053581684827804565 LKL 0.014685148373246193
epoch 19994 loss -0.0026334915310144424 LR -0.017249666154384613 LKL 0.01461617462337017
epoch 19995 loss 0.04222611337900162 LR 0.027577869594097137 LKL 0.01464824378490448
epoch 19996 loss 0.010589770041406155 LR -0.004039198160171509 LKL 0.014628968201577663
epoch 19997 loss 0.013569832779467106 LR -0.0010334476828575134 LKL 0.01460328046232462
epoch 19998 loss 0.002963623031973839 LR -0.011612825095653534 LKL 0.014576448127627373
epoch 19999 loss 0.019309919327497482 LR

In [52]:
#torch.save(encoder.state_dict(), 'encoderOUR_epoch_%d.pth' % (it))
#torch.save(decoder.state_dict(), 'decoderOUR_epoch_%d.pth' % (it))

In [100]:
encoder = Encoder(strokeSize, batch_size, N_he, N_hd, N_z).cuda()
decoder = Decoder(strokeSize, batch_size, N_he, N_hd, N_z, N_y).cuda()

saved_encoder = torch.load('encoderOUR_epoch_19999.pth')
saved_decoder = torch.load('decoderOUR_epoch_19999.pth')
encoder.load_state_dict(saved_encoder)
decoder.load_state_dict(saved_decoder)

inference_set = load_data['train']
inference_set = utils.DataLoader(
      inference_set,
      1,
      max_seq_length=max_seq_len)
normalizing_scale_factor = inference_set.calculate_normalizing_scale_factor()
inference_set.normalize(normalizing_scale_factor)
_, x, s = inference_set.random_batch() 

x = Variable(torch.from_numpy(x).type(torch.FloatTensor)).cuda()

encoder.train(False)
decoder.train(False)
h0, z, mu, sigma = encoder(x[:, 1:200+1, :])
y = decoder.predict(x[:, :200, :].contiguous(), h0[:,:N_hd].contiguous(), h0[:,N_hd:].contiguous(), z, M)

total images <= max_seq_len is 70000
[-0.14915210079271671, 0.11975969368868269, 1, 0, 0]
[-0.3572112779439145, 0.11264466957750406, 1, 0, 0]
[-0.44525912730916078, 0.014951914126096161, 1, 0, 0]
[-1.2534099204648264, -0.15539671124503712, 1, 0, 0]
[-0.68641260861935116, -0.36192482175583046, 1, 0, 0]
[-0.21461763250383589, -0.51983452885111459, 1, 0, 0]
[-0.078676019103115086, -0.72233939651925538, 1, 0, 0]
[0.2575702782064363, -0.7941813863239946, 1, 0, 0]
[0.35845989622737029, -0.5303003001702461, 1, 0, 0]
[0.4516947132108376, -0.26956052604297465, 1, 0, 0]
[0.38107966484968719, -0.095057607809660855, 1, 0, 0]
[0.49988485361466578, 0.099163742520244125, 1, 0, 0]
[0.6213365040557185, 0.61910179630773343, 1, 0, 0]
[0.43886155640207214, 0.76799999398042629, 1, 0, 0]
[0.11702464779260169, 0.75231744443615145, 1, 0, 0]
[0.01852544534681369, 0.76066827274818849, 1, 0, 0]
[-0.18859823637134771, 0.33350084773751132, 0, 1, 0]
[-0.69424636985836041, -2.9362873319114886, 1, 0, 0]
[0.1986421571

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:108: RuntimeWarning: covariance is not positive-semidefinite.


In [101]:
seq_x = [x[0] for x in y]
seq_y = [x[1] for x in y]
seq_z = [x[2]==0 for x in y]

In [102]:
def make_image(sequence,epoch, name='_output_'):
    strokes = np.split(sequence, np.where(sequence[:,2]>0)[0]+1)
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    for s in strokes:
        plt.plot(s[:,0],-s[:,1])
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    pil_image = PIL.Image.frombytes('RGB', canvas.get_width_height(),
                 canvas.tostring_rgb())
    name = str(epoch)+name+'.jpg'
    pil_image.save(name,"JPEG")
    plt.close("all")

In [103]:
import PIL
x_sample = np.cumsum(seq_x, 0)
y_sample = np.cumsum(seq_y, 0)
z_sample = np.array(seq_z)
sequence = np.stack([x_sample,y_sample,z_sample]).T
make_image(sequence,0.5)

In [81]:
np.random.choice(3, p=[.2,.3,.5])

2